In [1]:
import pickle
from GraphTsetlinMachine.graphs import Graphs
import numpy as np
with open('data.pkl', 'rb') as f:
    graphs_train, graphs_test, X_train, Y_train, X_test, Y_test = pickle.load(f)

In [2]:
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine, CommonTsetlinMachine
import numpy as np


In [ ]:
import optuna
from optuna.exceptions import TrialPruned

def objective(trial):
    # Suggest values for hyperparameters
    number_of_clauses = trial.suggest_int('number_of_clauses', 5000, 100000, step=5000)
    T = number_of_clauses * trial.suggest_float('T_factor', 0.5, 1.2)
    s = trial.suggest_float('s', 1.0, 10.0)
    depth = trial.suggest_int('depth', 5, 9, step=1)
    message_size = 32
    message_bits = 2
  
    epochs = 15

    # Initialize the Tsetlin machine with suggested hyperparameters
    tm = MultiClassGraphTsetlinMachine(
        number_of_clauses, T, s, depth=depth, message_size=message_size,
        message_bits=message_bits, number_of_state_bits=8, boost_true_positive_feedback=1,
    )

    for i in range(epochs):
        # Train the model
        tm.fit(graphs_train, Y_train, epochs=1, incremental=True)

        # Compute accuracy on test and training data
        result_test = 100 * (tm.predict(graphs_test) == Y_test).mean()
        result_train = 100 * (tm.predict(graphs_train) == Y_train).mean()

        # Report intermediate result and prune trial if not promising
        trial.report(result_test, i)

        if trial.should_prune():
            raise TrialPruned()

        # Early stopping conditions
        if result_test >= 100 or result_test < 65:
            break

    return result_test

# Create study and optimize with user-controlled stopping mechanism
study = optuna.create_study(
    direction="maximize",
    study_name="My_Custom_Study_15",  # Name your study
    storage="sqlite:///my_study.db",  # Save to SQLite database
    load_if_exists=True,  # If a study with this name already exists, load it
    pruner=optuna.pruners.MedianPruner()
) 

try:
    # This allows you to stop the optimization process manually
    study.optimize(objective, n_trials=100)
except KeyboardInterrupt:
    print("Optimization interrupted!")
    # Optionally, display the current best result when stopped
    print(f"Best result so far: {study.best_params}")


c:\Users\jonin\.conda\envs\hex\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-10-21 09:50:41,484] Using an existing study with name 'My_Custom_Study_15' instead of creating a new one.


Initialization of sparse structure.


c:\Users\jonin\.conda\envs\hex\Lib\site-packages\GraphTsetlinMachine\tm.py:189: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  mod_update = SourceModule(parameters + kernels.code_header + kernels.code_update, no_extern_c=True)
c:\Users\jonin\.conda\envs\hex\Lib\site-packages\GraphTsetlinMachine\tm.py:196: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  mod_evaluate = SourceModule(parameters + kernels.code_header + kernels.code_evaluate, no_extern_c=True)
